

## **Advances in Data Mining**

Stephan van der Putten | (s1528459) | stvdputtenjur@gmail.com  
Theo Baart | s2370328 | s2370328@student.leidenuniv.nl

### **Assignment 2**
This assignment is concerned with finding the set of similar users in the provided datasource. To be more explicit, in finding all pairs of users who have a Jaccard similarity of more than 0.5. Additionally, this assignment considers comparing the "naïve implementation" with the "LSH implementation". The "naïve implementation" can be found in the file `time_estimate.ipynb` and the "LSH implementation" in the file `lsh.ipynb`.

Note all implementations are based on the assignment guidelines and helper files given as well as the documentation of the used functions. Additionally, the following sources have been referenced and used as inspiration:
  1. CMSC643: Machine Learning and Data Mining: <http://www.hcbravo.org/dscert-mldm/projects/project_1/>
  2. Shared_Minhash_and_LSH_from_binned_date: <https://colab.research.google.com/drive/1HetBrWFRYqwUxn0v7wIwS7COBaNmusfD#scrollTo=hzPw8EMoW4i4&forceEdit=true&sandboxMode=true>


#### **LSH Implementation**
This notebook implements LSH in order to find all pairs of users with a Jaccard similarity of more than 0.5. As noted in the assignment instructions the data file is loaded from `user_movie.npy` and the list of user pairs are printed in the file `ans.txt`. Additionally, this implementation supports the setting of a random seed to determine the permutations to be used in LSH. The algorithm will continually save its output so as to aid in the evaluation criteria which only looks at the first 15 minutes of the LSH execution.
___

### **Helper Functions**
This section contains functions which aid and simplify the code for our LSH implementation
___

The following snippet handles all imports.

In [10]:
import time
import sys
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from collections import defaultdict
import scipy.optimize as opt
import math

The `compute_num_bands` function computes the optimal number of bands to use given a threshold and signature size. This is a slightly modified version of the `choose_nbands` function given in [1].

In order to do this the function uses the following parameters:
  * `t` - the desired threshold value
  * `s` - the size of the signature
  
The following command line arguments are expected:
  * `b` - the suggested number of bands to use
  * `final_t` - the computed threshold for this number of bands

In [12]:
def compute_num_bands(t, s):
    def error_fun(x):
        cur_t = (1/x[0])**(x[0]/s)
        return (t-cur_t)**2

    opt_res = opt.minimize(error_fun, x0=(10), method='Nelder-Mead')
    b = int(math.ceil(opt_res['x'][0]))
    r = round(n / b)
    final_t = (1/b)**(1/r)
    return b, final_t

The `generate_signature_hash_function` function generates a random hash function which returns a large numeric value to be used for hashing signatures into buckets. This is a slightly modified version of the `make_random_hash_fn` function given in [1].

In order to do this the function uses the following parameters:
  * `p` - largest value which can be randomly selected [default: 2^31 - 1]
  * `k` - the number of buckets to use [default: 12884901885]
  
The following command line arguments are expected:
  * `lambda` - a lambda function representing the random hash function

In [16]:
def generate_signature_hash_function(p=2**31-1, k=12884901885):#4294967295):
    a = np.random.randint(1,p-1)
    b = np.random.randint(0, p-1)
    return lambda x: ((a * x + b) % p) % k

### **LSHFunctions**
This section contains the functions which execute the various steps of the LSH algorithm.

___

The `generate_hash_function` function generates a random hash function which returns a large numeric value [i.e. the bucket]. This is a slightly modified version of the `make_random_hash_fn` function given in [1].

In order to do this the function uses the following parameters:
  * `p` - largest value which can be randomly selected [default: 2^31 - 1]
  * `k` - the number of buckets to use [default: 12884901885]
  
The following command line arguments are expected:
  * `lambda` - a lambda function representing the random hash function

In [2]:
user_movie = np.load('datasets/user_movie.npy')

In [ ]:
%%time
c = user_movie[:,0]
r = user_movie[:,1]
d = np.ones(len(c))
max_c = len(np.unique(c))
max_r = len(np.unique(r))
# m = csr_matrix((d, (r,c)), shape=(max_r, max_c))
csc = csc_matrix((d, (r,c)), shape=(max_r, max_c))
csr = csr_matrix((d, (r,c)), shape=(max_r, max_c))
signature_length = 50

# example = np.array([[1,0,0,1],[0,0,1,0],[0,1,0,1],[1,0,1,0],[0,0,1,0]])
# hash_func = np.array([[4,3,1,2,0], [3,0,4,2,1]])

In [100]:
example = np.array([[1,0,0,1,1,0,0,1],[0,0,1,0,0,0,1,0],[0,0,1,0,0,0,1,0],[0,1,0,1,1,0,0,1],[1,0,1,0,1,0,1,0],[0,1,1,1,1,1,0,1]])
hash_func = np.array([[5,4,3,1,2,0],[3,1,2,0,5,4],[1,2,0,5,4,3],[2,0,5,4,3,1],[0,5,4,3,1,2],[3,0,4,2,1,5],[0,4,2,1,5,3],[4,2,1,5,3,0],[2,1,5,3,0,4]])
c = csr_matrix(example)
s = rowminhash(9, hash_func, c)

In [101]:
display(c.todense())
display(hash_func)
display(s)

matrix([[1, 0, 0, 1, 1, 0, 0, 1],
        [0, 0, 1, 0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0, 0, 1, 0],
        [0, 1, 0, 1, 1, 0, 0, 1],
        [1, 0, 1, 0, 1, 0, 1, 0],
        [0, 1, 1, 1, 1, 1, 0, 1]], dtype=int32)

array([[5, 4, 3, 1, 2, 0],
       [3, 1, 2, 0, 5, 4],
       [1, 2, 0, 5, 4, 3],
       [2, 0, 5, 4, 3, 1],
       [0, 5, 4, 3, 1, 2],
       [3, 0, 4, 2, 1, 5],
       [0, 4, 2, 1, 5, 3],
       [4, 2, 1, 5, 3, 0],
       [2, 1, 5, 3, 0, 4]])

array([[2., 0., 0., 0., 0., 0., 2., 0.],
       [3., 0., 1., 0., 0., 4., 1., 0.],
       [1., 3., 0., 1., 1., 3., 0., 1.],
       [2., 1., 0., 1., 1., 1., 0., 1.],
       [0., 2., 1., 0., 0., 2., 1., 0.],
       [1., 2., 0., 2., 1., 5., 0., 2.],
       [0., 1., 2., 0., 0., 3., 2., 0.],
       [3., 0., 0., 0., 0., 0., 1., 0.],
       [0., 3., 0., 2., 0., 4., 0., 2.]])

In [3]:
def rowminhash(signature_length, hash_func, matrix):
    sigm = np.full((signature_length, matrix.shape[1]), np.inf)
    for row in range(matrix.shape[0]):
        ones = find(matrix[row, :])[1]
        hash = hash_func[:,row]
        B = sigm.copy()
        B[:,ones] = 1
        B[:,ones] = np.multiply(B[:,ones], hash.reshape((len(hash), 1)))
        sigm = np.minimum(sigm, B)
    return(sigm)

In [209]:
import random
from collections import defaultdict
def lsh_r_bucket_to_id(sigm,id,b,r,numhashes):
    id = np.array(id)
    number_of_users = sigm.shape[1]
    hash_buckets = defaultdict(list)
    hf = make_random_hash_fn()
    t1 = time.time()    
    for i in range(number_of_users):
        if i % 10000==0:
            print(str(round(100*i/number_of_users,2))+' percent complete in '+str(round(time.time()-t1,2))+ ' seconds')
#         row = u[i,:]  
        row = sigm[:,i] 
        for j in range(b):
            r_signature = str(row[j*r:(j+1)*r])
            r_hash = hash(r_signature)
#             print(r_signature)
#             print(r_hash)
            r_hash = hf(r_hash)
            hash_buckets[r_hash].append(id[i])
    hash_buckets_set = {k: set(v) for k,v in hash_buckets.items()}
    return hash_buckets_set

In [166]:
sigm100 = np.load('datasets/sign_matrix_100.npy')
sigm50 = np.load('datasets/sign_matrix.npy')

In [169]:
sigm = sigm100 
# sigm = sigm50
threshold=0.57 # overshoot so as to get a similarity matrix closer to 0.5
numhashes = sigm.shape[0]
# b, _ = choose_nbands(threshold, numhashes)
b = 6
r = 15
r = round(numhashes / b)
threshold = (1/b)**(1/r)
print(b*r,'vs',numhashes)
print(threshold,b,r)
user_ids = np.array(list(range(sigm.shape[1])))
buckets = lsh_r_bucket_to_id(sigm,user_ids,b,r,numhashes)

100 vs 100
0.5492802716530588 20 5
0.0 percent complete in 0.0 seconds
9.64 percent complete in 30.26 seconds
19.29 percent complete in 62.77 seconds
28.93 percent complete in 102.01 seconds
38.57 percent complete in 136.38 seconds
48.21 percent complete in 161.7 seconds
57.86 percent complete in 186.94 seconds
67.5 percent complete in 214.16 seconds
77.14 percent complete in 238.86 seconds
86.79 percent complete in 265.06 seconds
96.43 percent complete in 289.54 seconds


3503486605907668772


-3378229029050999201

In [170]:
%%time
import itertools as it
pairs = set()
max_l = 0
print('no_buckets:',len(buckets))
short_buckets = {k: v for k, v in buckets.items() if len(v) >= 2}
print('no_candidate_buckets',len(short_buckets))
i = 0
for v in short_buckets.values():
    if len(v) > max_l:
        max_l = len(v)
    pairs.update(set(it.combinations(v,2)))
#         i += 1
#         if i == 100:
#             break
print('max_bucket_size:',max_l)
print('no_pairs:',len(pairs))
print('pairs:',pairs)

no_buckets: 1193619
no_candidate_buckets 191783
max_bucket_size: 0
no_pairs: 0
pairs: set()
Wall time: 714 ms


In [ ]:
sb = short_buckets[4874]
print(len(sb),sb)
sb = list(sb)
base = sigm[:,sb[0]]
# print(base)
for i in range(1,len(sb)):
    far = sigm[:,sb[i]]
    intersect = np.intersect1d(base,far)
    union = np.union1d(base,far)
    print(i,len(intersect)/len(union))
#     print(sigm[84:91,i])
# print(sigm[84:91,905])
# print(sigm[84:91,3831])
# print(sigm[84:91,5018])
# print(sigm[84:91,23663])
# print(sigm[84:91,23490:23500].T)

In [ ]:
np.random.seed = 42

# example = csr
# example = np.array([[1,0,0,1],[0,0,1,0],[0,1,0,1],[1,0,1,0],[0,0,1,0]])
# %time sigm1 = minhash(signature_length,hash_func, example)
# signature_length = 100
# hash_func = np.array([np.random.permutation(csr.shape[0]) for i in range(signature_length)])
# %time sigm1 = rowminhash(signature_length ,hash_func, csr)

signature_length = 50
hash_func = np.array([np.random.permutation(csr.shape[0]) for i in range(signature_length)])
%time sigm2 = rowminhash(signature_length,hash_func, csr)
# print(sigm2)
# np.save('datasets/sign_matrix_100', sigm1)
# np.save('datasets/sign_matrix', sigm2)

In [ ]:
#TODO write out our results ans.txt 
#TODO set seed in our perm hash function
#TODO Find pairs of similar users from buckets
#TODO Elegance aka classes, comments/report/citation
#TODO IMPROVE EFFICIENCY FOR LONGER SIGNATURES Time < 15

### **Program Execution**
This section is concerned with parsing the input arguments and determining the execution flow of the program.

___
The `main` function handles the start of execution from the command line.

In order to do this the function uses the following parameters:
  * `argv` - the command line arguments given to the program
  
The following command line arguments are expected:
  * `seed` - the value to use as random seed
  * `path` - the location of the `user_movies.npy` file

In [ ]:
def main(argv):
    seed = sys.argv[1]
    path = sys.argv[2]
    print(seed, path)

The following snippet passes the start of the program and the command line arguments to the `main` function.

In [ ]:
if __name__ == "__main__":
    main(sys.argv[1:])